In [1]:
!pip install llama-index==0.9.19 loguru -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [77]:
from loguru import logger
import json
from typing import Sequence, List, Dict, Any

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.llms.ollama import Ollama
from llama_index.core.tools.types import BaseTool, ToolOutput
from llama_index.core.tools.function_tool import FunctionTool
from llama_index.core.base.llms.types  import ChatMessage, MessageRole, ChatResponse
from llama_index.core.prompts.base  import ChatPromptTemplate
from openai.types.chat import ChatCompletionMessageToolCall
from rich.pretty import pprint
import nest_asyncio

nest_asyncio.apply()

## Tools

In [82]:
def int_mult(a: int, b: int) -> int:
    """Apply a * b and returns the result as int"""
    res = a * b
    logger.debug(f"Apply {a} * {b} and returns {res}")

    return res

def int_add(a: int, b: int) -> int:
    """Apply a + b and returns the result as int"""
    res = a + b
    logger.debug(f"Apply {a} + {b} and returns {res}")

    return res


def random_joke()->str:
    """A crazy creator that can create a random joke."""
    #model: OpenAI = OpenAI(temperature=1.5)
    model = Ollama(model="gemma:2b-instruct", temperature=1.5)
    res = model.complete(f"You are a crazy creator that can create a random joke.")
    return res.text.strip()


def translate(string: str)->str:
    """Translate the string into Chinese."""
    #model: OpenAI = OpenAI(temperature=0)
    model = Ollama(model="gemma:2b-instruct", temperature=0)
    res = model.complete(f"Translate {string} into Chinese, only return the result.")
    return res.text.strip()

In [84]:
a_joke = random_joke()
joke_translated = translate(a_joke)
pprint(joke_translated)

'结果，他获得了奖项。 (zì jìng tā wèi yīn de shì, zé yú wèi yǐn diàn de shì)'

### Convert Python functiont to function-tool.

Similar to `@tool` in LangChain

In [85]:
int_mult_tool, int_add_tool = FunctionTool.from_defaults(fn=int_mult), FunctionTool.from_defaults(fn=int_add)
int_tools: Sequence[BaseTool]  = [int_mult_tool, int_add_tool]

pprint(int_tools)

[
│   <llama_index.core.tools.function_tool.FunctionTool object at 0x138c1ead0>,
│   <llama_index.core.tools.function_tool.FunctionTool object at 0x138ee5350>
]

In [86]:
for tool in int_tools:
    pprint(tool.metadata)
    print("-----")

ToolMetadata(
│   description='int_mult(a: int, b: int) -> int\nApply a * b and returns the result as int',
│   name='int_mult',
│   fn_schema=<class 'pydantic.v1.main.int_mult'>
)

-----


ToolMetadata(
│   description='int_add(a: int, b: int) -> int\nApply a + b and returns the result as int',
│   name='int_add',
│   fn_schema=<class 'pydantic.v1.main.int_add'>
)

-----


In [87]:
tools_dict: Dict[str, FunctionTool] = {tool.metadata.name: tool for tool in int_tools}
pprint(tools_dict)

{
│   'int_mult': <llama_index.core.tools.function_tool.FunctionTool object at 0x138c1ead0>,
│   'int_add': <llama_index.core.tools.function_tool.FunctionTool object at 0x138ee5350>
}

### Call a tool

In [88]:
int_mult_tool(**{"a":12,"b":34})

2024-02-28 22:06:23.858 | DEBUG    | __main__:int_mult:4 - Apply 12 * 34 and returns 408


ToolOutput(content='408', tool_name='int_mult', raw_input={'args': (), 'kwargs': {'a': 12, 'b': 34}}, raw_output=408)

## Model

In [89]:
model: OpenAI = OpenAI(temperature=0, model="gpt-4-1106-preview")

## Prompt template

Ref: https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts.html



In [90]:
messages: Sequence[ChatMessage] = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=("You are an assisant to perform the user input."),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=("{input}"),
    ),
]
tmpl: ChatPromptTemplate = ChatPromptTemplate(messages)

logger.debug("in string")

messages_str: str = tmpl.format(input="hi, template")
pprint(messages_str)

logger.debug("in list")

messages: Sequence[ChatMessage] = tmpl.format_messages(input="hi, messages")
pprint(messages)

2024-02-28 22:06:23.878 | DEBUG    | __main__:<module>:13 - in string


'system: You are an assisant to perform the user input.\nuser: hi, template\nassistant: '

2024-02-28 22:06:23.882 | DEBUG    | __main__:<module>:18 - in list


[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(role=<MessageRole.USER: 'user'>, content='hi, messages', additional_kwargs={})
]

## Query with tools

In [91]:
messages: Sequence[ChatMessage]  = tmpl.format_messages(input="What is the result of the 12*34?")
pprint(messages)

[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.USER: 'user'>,
│   │   content='What is the result of the 12*34?',
│   │   additional_kwargs={}
│   )
]

### Notice

Cannot call `model.chat(messages, tools=int_tools)`, it complaints, because it is not OpenAI format.

```
TypeError                                 Traceback (most recent call last)
<ipython-input-26-76230e6dd870> in <cell line: 1>()
----> 1 res = model.chat(messages, tools=int_tools)
      2 pprint(res)

16 frames
/usr/lib/python3.10/json/encoder.py in default(self, o)
    177
    178         """
--> 179         raise TypeError(f'Object of type {o.__class__.__name__} '
    180                         f'is not JSON serializable')
    181

TypeError: Object of type FunctionTool is not JSON serializable
```



### Convert to OpenAI function

Similar to LangChain `langchain_core.utils.function_calling.convert_to_openai_function`

In [92]:
openai_tools: Dict[str, Any] = [
            tool.metadata.to_openai_tool() for tool in int_tools
        ]
pprint(openai_tools)

[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'int_mult',
│   │   │   'description': 'int_mult(a: int, b: int) -> int\nApply a * b and returns the result as int',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}},
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'int_add',
│   │   │   'description': 'int_add(a: int, b: int) -> int\nApply a + b and returns the result as int',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}},
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
]

### Useful query

In [93]:
res:ChatResponse = model.chat(messages, tools=openai_tools)
pprint(res)

ChatResponse(
│   message=ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_TVaa1e1iySBfj4PqTaBcpzBa',
│   │   │   │   │   function=Function(arguments='{"a":12,"b":34}', name='int_mult'),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   raw={
│   │   'id': 'chatcmpl-8xLQ8FR0mr2mkgdvYMQOC3i4U6tNv',
│   │   'choices': [
│   │   │   Choice(
│   │   │   │   finish_reason='tool_calls',
│   │   │   │   index=0,
│   │   │   │   logprobs=None,
│   │   │   │   message=ChatCompletionMessage(
│   │   │   │   │   content=None,
│   │   │   │   │   role='assistant',
│   │   │   │   │   function_call=None,
│   │   │   │   │   tool_calls=[
│   │   │   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   │   │   id='call_TVaa1e1iySBfj4PqTaBcpzBa',
│   │   │   │   │   │   │   function=Function(arguments='{"a":12,"b":34}', name='int_mult'),
│   │   │   │   │   │   │   type='function'
│   │   │   │   │   │   )
│   │   │   │   │   ]
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   'created': 1709154384,
│   │   'model': 'gpt-4-1106-preview',
│   │   'object': 'chat.completion',
│   │   'system_fingerprint': 'fp_7cc080b25b',
│   │   'usage': CompletionUsage(completion_tokens=18, prompt_tokens=148, total_tokens=166)
│   },
│   delta=None,
│   additional_kwargs={}
)

### Useless query, no tool required

In [94]:
messages: Sequence[ChatMessage]  = tmpl.format_messages(input="hi")
pprint(messages)

[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(role=<MessageRole.USER: 'user'>, content='hi', additional_kwargs={})
]

In [95]:
res: ChatResponse = model.chat(messages, tools=openai_tools)
pprint(res)

ChatResponse(
│   message=ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content='Hello! How can I assist you today?',
│   │   additional_kwargs={}
│   ),
│   raw={
│   │   'id': 'chatcmpl-8xLQ9TZLXQsXTY2J8tt16tl2uWlUn',
│   │   'choices': [
│   │   │   Choice(
│   │   │   │   finish_reason='stop',
│   │   │   │   index=0,
│   │   │   │   logprobs=None,
│   │   │   │   message=ChatCompletionMessage(
│   │   │   │   │   content='Hello! How can I assist you today?',
│   │   │   │   │   role='assistant',
│   │   │   │   │   function_call=None,
│   │   │   │   │   tool_calls=None
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   'created': 1709154385,
│   │   'model': 'gpt-4-1106-preview',
│   │   'object': 'chat.completion',
│   │   'system_fingerprint': 'fp_dbe4f8ca8b',
│   │   'usage': CompletionUsage(completion_tokens=10, prompt_tokens=138, total_tokens=148)
│   },
│   delta=None,
│   additional_kwargs={}
)

## Agent loop

In [96]:
class AgentExecutor:
    def __init__(self, tools:Sequence[Any]):
        func_tools: Sequence[BaseTool]  = list(map(lambda f: FunctionTool.from_defaults(fn=f), tools ))
        self.func_tools_dict: Dict[str, FunctionTool] = {tool.metadata.name: tool for tool in func_tools}
        self.openai_tools: Sequence[Dict[str, Any]] = list(map(lambda func_tool: func_tool.metadata.to_openai_tool(), func_tools))

        self.chat_history: List[ChatMessage] = list()

        messages: Sequence[ChatMessage] = [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=("You are an assisant to perform the user input."),
            ),
            ChatMessage(
                role=MessageRole.USER,
                content=("{input}"),
            ),
        ]
        self.chat_prompt_template: ChatPromptTemplate = ChatPromptTemplate(messages)

        self.model: OpenAI = OpenAI(temperature=0, model="gpt-4-1106-preview")

    def should_continue(self, chat_response: ChatResponse):
        return chat_response.message.additional_kwargs.get("tool_calls", None) is not None

    def run_tool(self, tool_call: ChatCompletionMessageToolCall) -> ChatMessage:
        logger.info("Running tool")
        func_id: str = tool_call.id
        func_name: str = tool_call.function.name
        args_json: str = tool_call.function.arguments

        func: FunctionTool = self.func_tools_dict.get(func_name)
        res: ToolOutput = func(**json.loads(args_json))
        pprint(res)
        return ChatMessage(
            role = MessageRole.TOOL,
            content=str(res),
            name=func_name,
            additional_kwargs={
                "tool_call_id": func_id,
                "name": func_name,
            }
        )

    def __call__(self, human_input: str) -> ChatMessage:
        logger.info("Agent on start.")
        messages: Sequence[ChatMessage] = self.chat_prompt_template.format_messages(input=human_input)
        self.chat_history.extend(messages)
        while True:
            chat_response: ChatResponse = self.model.chat(messages, tools=self.openai_tools)
            ai_message: ChatMessage = chat_response.message
            logger.info("AI messsaging")
            pprint(ai_message)
            if not self.should_continue(chat_response):
                self.chat_history.extend([ai_message])
                logger.info("Agent on stop.")
                return ai_message
            else:
                func_message = self.run_tool(ai_message.additional_kwargs["tool_calls"][0])
                self.chat_history.extend([ai_message, func_message])
                messages = self.chat_history
                logger.info("Agent in continue.")

    def __str__(self) -> str:
        print("func_tools_dict")
        pprint(self.func_tools_dict)
        print("openai_tools")
        pprint(self.openai_tools)
        print("chat_prompt_template")
        pprint(self.chat_prompt_template)
        print("chat_history")
        pprint(self.chat_history)

agent_exe: AgentExecutor = AgentExecutor(
    tools = [random_joke, translate]
)


### Run agent

In [97]:
#@title { vertical-output: true}

human_input: str = """
Please create a random joke and provide the translated result
without including the original language, newlines, empty spaces, or instructions.
"""

res: ChatMessage = agent_exe(human_input=human_input)
print("\n---Final result---")
pprint(res)

2024-02-28 22:06:26.754 | INFO     | __main__:__call__:46 - Agent on start.
2024-02-28 22:06:29.604 | INFO     | __main__:__call__:52 - AI messsaging


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content=None,
│   additional_kwargs={
│   │   'tool_calls': [
│   │   │   ChatCompletionMessageToolCall(
│   │   │   │   id='call_nSlkkXbFcnhkuqLssRnNDAIn',
│   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   type='function'
│   │   │   )
│   │   ]
│   }
)

2024-02-28 22:06:29.609 | INFO     | __main__:run_tool:27 - Running tool


ToolOutput(
│   content="Sure, here's a random joke for you:\n\nWhat do you call a boomerang that won't come back?\n\nA stick.",
│   tool_name='random_joke',
│   raw_input={'args': (), 'kwargs': {}},
│   raw_output="Sure, here's a random joke for you:\n\nWhat do you call a boomerang that won't come back?\n\nA stick."
)

2024-02-28 22:06:35.144 | INFO     | __main__:__call__:62 - Agent in continue.
2024-02-28 22:06:37.137 | INFO     | __main__:__call__:52 - AI messsaging


...

2024-02-28 22:06:41.912 | INFO     | __main__:__call__:62 - Agent in continue.
2024-02-28 22:06:44.503 | INFO     | __main__:__call__:52 - AI messsaging


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content='你怎么称呼一个不会回来的回旋镖？一根棍子。',
│   additional_kwargs={}
)

2024-02-28 22:06:44.507 | INFO     | __main__:__call__:56 - Agent on stop.



---Final result---


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content='你怎么称呼一个不会回来的回旋镖？一根棍子。',
│   additional_kwargs={}
)

### Agent history

In [98]:
agent_exe.__str__()

func_tools_dict


{
│   'random_joke': <llama_index.core.tools.function_tool.FunctionTool object at 0x1385d6450>,
│   'translate': <llama_index.core.tools.function_tool.FunctionTool object at 0x138f035d0>
}

openai_tools


[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'random_joke',
│   │   │   'description': 'random_joke() -> str\nA crazy creator that can create a random joke.',
│   │   │   'parameters': {'type': 'object', 'properties': {}}
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'translate',
│   │   │   'description': 'translate(string: str) -> str\nTranslate the string into Chinese.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'string': {'title': 'String', 'type': 'string'}},
│   │   │   │   'required': ['string']
│   │   │   }
│   │   }
│   }
]

chat_prompt_template


ChatPromptTemplate(
│   metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>},
│   template_vars=['input'],
│   kwargs={},
│   output_parser=None,
│   template_var_mappings=None,
│   function_mappings=None,
│   message_templates=[
│   │   ChatMessage(
│   │   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   │   content='You are an assisant to perform the user input.',
│   │   │   additional_kwargs={}
│   │   ),
│   │   ChatMessage(role=<MessageRole.USER: 'user'>, content='{input}', additional_kwargs={})
│   ]
)

chat_history


[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.USER: 'user'>,
│   │   content='\nPlease create a random joke and provide the translated result\nwithout including the original language, newlines, empty spaces, or instructions.\n',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_nSlkkXbFcnhkuqLssRnNDAIn',
│   │   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   ChatMessage(
│   │   role=<MessageRole.TOOL: 'tool'>,
│   │   content="Sure, here's a random joke for you:\n\nWhat do you call a boomerang that won't come back?\n\nA stick.",
│   │   additional_kwargs={'tool_call_id': 'call_nSlkkXbFcnhkuqLssRnNDAIn', 'name': 'random_joke'}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_qwxrrJBvlqhurT3gUtbXYbUF',
│   │   │   │   │   function=Function(
│   │   │   │   │   │   arguments='{"string":"What do you call a boomerang that won\'t come back? A stick."}',
│   │   │   │   │   │   name='translate'
│   │   │   │   │   ),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   ChatMessage(
│   │   role=<MessageRole.TOOL: 'tool'>,
│   │   content="A stick.\n\nA boomerang that won't come back is called a stick.",
│   │   additional_kwargs={'tool_call_id': 'call_qwxrrJBvlqhurT3gUtbXYbUF', 'name': 'translate'}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content='你怎么称呼一个不会回来的回旋镖？一根棍子。',
│   │   additional_kwargs={}
│   )
]

## OpenAIAgent

This is Llama-Index buildin, it encapsulates a lot details of Agent running and very robust.

In [99]:
from llama_index.agent.openai import OpenAIAgent

func_tools: Sequence[BaseTool]  = list(map(lambda f: FunctionTool.from_defaults(fn=f), [random_joke, translate] ))
openai_agent = OpenAIAgent.from_tools(
    tools=func_tools,
    llm=model,
    verbose=True
)
res = openai_agent.chat(human_input)
print("\nFinal result")
pprint(res)

Added user message to memory: 
Please create a random joke and provide the translated result
without including the original language, newlines, empty spaces, or instructions.

=== Calling Function ===
Calling function: random_joke with args: {}
Got output: What do you call a joke that's too long?

A long shot!

=== Calling Function ===
Calling function: translate with args: {"string":"What do you call a joke that's too long?\n\nA long shot!"}
Got output: 很漫长！


Final result


AgentChatResponse(
│   response='很漫长！',
│   sources=[
│   │   ToolOutput(
│   │   │   content="What do you call a joke that's too long?\n\nA long shot!",
│   │   │   tool_name='random_joke',
│   │   │   raw_input={'args': (), 'kwargs': {}},
│   │   │   raw_output="What do you call a joke that's too long?\n\nA long shot!"
│   │   ),
│   │   ToolOutput(
│   │   │   content='很漫长！',
│   │   │   tool_name='translate',
│   │   │   raw_input={
│   │   │   │   'args': (),
│   │   │   │   'kwargs': {'string': "What do you call a joke that's too long?\n\nA long shot!"}
│   │   │   },
│   │   │   raw_output='很漫长！'
│   │   )
│   ],
│   source_nodes=[]
)